In [1]:
import pymrio
exio3 = pymrio.parse_exiobase3(path="exiobase/IOT_2022_pxp.zip")
import country_converter as coco
cc = coco.CountryConverter()
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Suppress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
exio3.calc_all()
emptyY = exio3.Y.copy()
emptyY.iloc[:,:] = 0

In [4]:
# Setting up the variables
vehicles_list = ["ICEV","PHEV","BEV","FCEV"]
stressor = "Carbon dioxide (CO2) IPCC categories 1 to 4 and 6 to 7 (excl land use, land use change and forestry)"
scenario_list = ["Projection-APS"]
year_list = [2030]
demand_list = ['normal']
demand = demand_list[0]
column_list = ["product","vehicles","stage","region","sector","emission"]
stage = "manufacturing"
scenario = scenario_list[0]

In [5]:
def calc_emission(vehicles,country,outlook): 
    demandY = emptyY.copy()
    
    demandY["NL",vehicles] = outlook[vehicles] ## NL as a place holder for european demand, we donot concern on which country the EV will be consumed, only the aggregated EU demand
    demandY.fillna(0,inplace=True)
    
    exio3.Y = demandY
    
    # calculate with new demand
    # coeff : A, L, S, M should be the same with original
    exio3.x = None # reset the output
    exio3.Z = None  # expect new transaction to satisfy the new the output
    exio3.impacts.D_cba = None # reset foot print cba, others are not synch because they consider the emission from final demand
    exio3.impacts.F = None
    exio3.calc_all()
     
    diag_emission = None
    #become a matrix F of regional - sector X regional - sector 
    diag_emission = exio3.impacts.diag_stressor(stressor)
    diag_emission.F
    
    exio3.emission  = diag_emission
    exio3.calc_all()
    
    #print(exio3.meta)
    print("done calculation for "+vehicles+" product of "+country)
    emission = exio3.emission.D_cba
    return emission

In [6]:
# read new outlook 
file_path = "data/outlook_footprint_"+stage+"_country.csv"

print(file_path)
#---------------------------------
outlook_all = pd.read_csv(file_path)

country_list = outlook_all["region"].unique().tolist()
country_list

data/outlook_footprint_manufacturing_country.csv


['CN', 'CZ', 'DE', 'FR', 'IT', 'KR', 'RO', 'SE', 'US']

In [7]:
outlook_pivot = outlook_all.pivot_table(index=["region","sector"],columns="vehicles",values="value")

In [10]:
#start collectimg emission data
import gc

for country in country_list:
    emission_country = pd.DataFrame(columns=column_list)
    outlook = outlook_pivot[outlook_pivot.index.get_level_values("region")==country]
        
    for vehicles in vehicles_list:
        emission = calc_emission(vehicles, country, outlook)
        emission_regions_total = emission.sum(axis=1).to_frame().reset_index()
        emission_regions_total.rename(columns = {0:"emission"}, inplace=True)
        emission_regions_total["stage"] = stage
        emission_regions_total["vehicles"] =  vehicles
        emission_regions_total["product"] =  country
        emission_country = pd.concat([emission_country,emission_regions_total],axis=0) 
        del emission_regions_total
        gc.collect()
    
    emission_country.to_csv("outputs/emission_product_country_"+country+".csv",index=False)    
        
#emission_stage.reset_index(drop=True,inplace=True) 

done calculation for ICEV product of CN
done calculation for PHEV product of CN
done calculation for BEV product of CN
done calculation for FCEV product of CN
done calculation for ICEV product of CZ
done calculation for PHEV product of CZ
done calculation for BEV product of CZ
done calculation for FCEV product of CZ
done calculation for ICEV product of DE
done calculation for PHEV product of DE
done calculation for BEV product of DE
done calculation for FCEV product of DE
done calculation for ICEV product of FR
done calculation for PHEV product of FR
done calculation for BEV product of FR
done calculation for FCEV product of FR
done calculation for ICEV product of IT
done calculation for PHEV product of IT
done calculation for BEV product of IT
done calculation for FCEV product of IT
done calculation for ICEV product of KR
done calculation for PHEV product of KR
done calculation for BEV product of KR
done calculation for FCEV product of KR
done calculation for ICEV product of RO
done c

In [ ]:
emission_stage = pd.DataFrame(columns=column_list)

for country in country_list:
    emission_regions_total. = pd.read_csv("outputs/emission_product_country_"+country+".csv")
    emission_stage = pd.concat([emission_stage,emission_regions_total],axis=0) 

emission_stage

In [13]:
import os
file_path = "outputs/emission_product_all_country.csv" 
try:
    os.remove(file_path)
except FileNotFoundError:
    print(f"{file_path} not yet exist.")

for country in country_list:
    file_path_country = "outputs/emission_product_country_" + country + ".csv"
    emission_country = pd.read_csv(file_path_country)
    header_bool = not os.path.exists(file_path)
    emission_country.to_csv(file_path, mode='a', index=False, header=header_bool)
    